## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [17]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [18]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_20.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_20.csv')

DI_train = pd.read_csv('../data/DI_train_dataset_20.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_20.csv')

In [19]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

### 인코딩 

In [20]:
IVF_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "임신_시도_또는_마지막_임신_경과_연수",
    "배란_유도_유형",
    "배아_생성_주요_이유",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "변환된_특정_시술_유형",
    "채취_해동_차이",
    "해동_혼합_차이",
    "혼합_이식_차이",
    "이식_해동_차이"
]

In [21]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "임신_시도_또는_마지막_임신_경과_연수",
    "정자_기증자_나이",
    "변환된_특정_시술_유형"
]

In [22]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

In [23]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

## Modeling

train 데이터의 일부를 검증데이터로 사용하여 성능지표 출력

In [25]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_20.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_20.csv')
DI_train = pd.read_csv('../data/DI_train_dataset_20.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_20.csv')

# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']
DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

# 모든 범주형 변수를 문자열로 변환
IVF_categorical_columns = IVF_X.select_dtypes(include=['object']).columns.tolist()
DI_categorical_columns = DI_X.select_dtypes(include=['object']).columns.tolist()

IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

# K-Fold를 사용하여 IVF와 DI의 잘못된 예측 데이터 수집
kf = KFold(n_splits=5, shuffle=True, random_state=42)

IVF_wrong_predictions = []
DI_wrong_predictions = []

# K-Fold 교차 검증을 통한 잘못된 예측 수집 함수
def collect_wrong_predictions(X, y, wrong_predictions, model_params):
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMClassifier(**model_params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        wrong_idx = val_idx[y_pred != y_val.values]
        wrong_predictions.append(X.iloc[wrong_idx])

# IVF와 DI에 대해 잘못된 예측 수집
IVF_model_params = {
    'n_estimators': 4471,
    'num_leaves': 13,
    'max_depth': 279,
    'learning_rate': 0.007075124517450591,
    'min_child_samples': 26,
    'subsample': 0.29772991936701476,
    'colsample_bytree': 0.8913054521763838,
    'reg_alpha': 0.0004860363321690653,
    'reg_lambda': 311.08056657247363,
    'min_split_gain': 0.18214905183450955,
    'random_state': 42,
    'boosting_type': 'gbdt',
    'verbose': -1
}

DI_model_params = {
    'n_estimators': 1816,
    'num_leaves': 3926,
    'max_depth': 259,
    'learning_rate': 0.00238377640011148,
    'min_child_samples': 1,
    'subsample': 0.7610056627240331,
    'colsample_bytree': 0.6655579164853634,
    'reg_alpha': 0.00025227758337188327,
    'reg_lambda': 76.744107215122684,
    'min_split_gain': 0.007773520329665474,
    'random_state': 42,
    'boosting_type': 'gbdt',
    'verbose': -1
}

collect_wrong_predictions(IVF_X, IVF_y, IVF_wrong_predictions, IVF_model_params)
collect_wrong_predictions(DI_X, DI_y, DI_wrong_predictions, DI_model_params)

# 잘못된 예측 데이터 결합
wrong_data = pd.concat(IVF_wrong_predictions + DI_wrong_predictions)

# 오토인코더 정의
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 오토인코더 학습
input_dim = wrong_data.shape[1]
autoencoder = Autoencoder(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.01)

data_tensor = torch.tensor(wrong_data.values, dtype=torch.float32)
dataloader = DataLoader(TensorDataset(data_tensor), batch_size=32, shuffle=True)

for epoch in range(50):
    for batch in dataloader:
        optimizer.zero_grad()
        output = autoencoder(batch[0])
        loss = criterion(output, batch[0])
        loss.backward()
        optimizer.step()

# 테스트 데이터 보정
all_test_data = pd.concat([IVF_test, DI_test])
test_tensor = torch.tensor(all_test_data.drop(['ID'], axis=1).values, dtype=torch.float32)
predicted_output = autoencoder(test_tensor).detach().numpy()
reconstruction_error = np.mean((all_test_data.drop(['ID'], axis=1).values - predicted_output) ** 2, axis=1)
threshold = np.percentile(reconstruction_error, 95)

# LGBM 예측 수행
IVF_model = lgb.LGBMClassifier(**IVF_model_params)
IVF_model.fit(IVF_X, IVF_y)
IVF_pred = IVF_model.predict(IVF_test.drop('ID', axis=1))

DI_model = lgb.LGBMClassifier(**DI_model_params)
DI_model.fit(DI_X, DI_y)
DI_pred = DI_model.predict(DI_test.drop('ID', axis=1))

# 최종 예측 수행
final_predictions = np.concatenate([IVF_pred, DI_pred])

# 재구성 오류가 임계값을 초과하는 경우 예측 변경
final_predictions[reconstruction_error > threshold] = 1 - final_predictions[reconstruction_error > threshold]

# 검증 데이터에 대한 성능 평가
IVF_val_pred = IVF_model.predict(IVF_X) 
DI_val_pred = DI_model.predict(DI_X)    

# 성능 지표 계산
IVF_accuracy = accuracy_score(IVF_y, IVF_val_pred)
IVF_f1 = f1_score(IVF_y, IVF_val_pred)
IVF_auc = roc_auc_score(IVF_y, IVF_model.predict_proba(IVF_X)[:, 1])

DI_accuracy = accuracy_score(DI_y, DI_val_pred)
DI_f1 = f1_score(DI_y, DI_val_pred)
DI_auc = roc_auc_score(DI_y, DI_model.predict_proba(DI_X)[:, 1])

# 최종 병합된 예측에 대한 성능 지표 계산
merged_pred = np.concatenate([IVF_val_pred, DI_val_pred])
merged_y = np.concatenate([IVF_y, DI_y])

merged_accuracy = accuracy_score(merged_y, merged_pred)
merged_f1 = f1_score(merged_y, merged_pred)
merged_auc = roc_auc_score(merged_y, np.concatenate([IVF_model.predict_proba(IVF_X)[:, 1], DI_model.predict_proba(DI_X)[:, 1]]))

print("--- IVF Model Performance ---")
print(f"Accuracy: {IVF_accuracy}, F1 Score: {IVF_f1}, AUC: {IVF_auc}")

print("--- DI Model Performance ---")
print(f"Accuracy: {DI_accuracy}, F1 Score: {DI_f1}, AUC: {DI_auc}")

print("--- Merged Model Performance ---")
print(f"Accuracy: {merged_accuracy}, F1 Score: {merged_f1}, AUC: {merged_auc}")


--- IVF Model Performance ---
Accuracy: 0.7463047686081296, F1 Score: 0.20898537351771265, AUC: 0.7443415999629354
--- DI Model Performance ---
Accuracy: 0.8710651828298888, F1 Score: 0.0, AUC: 0.742958485813674
--- Merged Model Performance ---
Accuracy: 0.7493660812508289, F1 Score: 0.2068931463559155, AUC: 0.7464352722423071


test 데이터 전부 사용해서

In [31]:
# IVF 데이터에 대한 최종 모델 학습 및 예측
IVF_model_final = lgb.LGBMClassifier(**IVF_model_params)
IVF_model_final.fit(IVF_X, IVF_y)
IVF_test_pred = IVF_model_final.predict(IVF_test.drop('ID', axis=1))

# DI 데이터에 대한 최종 모델 학습 및 예측
DI_model_final = lgb.LGBMClassifier(**DI_model_params)
DI_model_final.fit(DI_X, DI_y)
DI_test_pred = DI_model_final.predict(DI_test.drop('ID', axis=1))

# 최종 제출 파일 생성
IVF_submission = pd.DataFrame({'ID': IVF_test['ID'], 'probability': IVF_test_pred})
DI_submission = pd.DataFrame({'ID': DI_test['ID'], 'probability': DI_test_pred})

submission = pd.concat([IVF_submission, DI_submission])
submission.to_csv('../submission/code21_1_submit.csv', index=False)

print("제출 파일(code21_1_submit.csv) 생성 완료!")

제출 파일(code21_1_submit.csv) 생성 완료!


In [32]:
import pandas as pd

# 두 CSV 파일을 읽어옵니다.
df1 = pd.read_csv('../submission/code21_1_submit.csv')
df2 = pd.read_csv('../submission/code21_submit.csv')

# '임신_성공_여부' 열을 비교합니다.
comparison = df1['probability'] == df2['probability']

# 차이점이 있는 행을 출력합니다.
differences = df1[~comparison]

len(differences)

0

In [33]:
import pandas as pd

# 두 CSV 파일을 읽어옵니다.
df1 = pd.read_csv('../submission/code21_1_submit.csv')
df2 = pd.read_csv('../submission/code20_submit.csv')

# '임신_성공_여부' 열을 비교합니다.
comparison = df1['probability'] == df2['probability']

# 차이점이 있는 행을 출력합니다.
differences = df1[~comparison]

len(differences)

90067

데이콘 PUBLIC 0.5422133005

.